In [149]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import gensim, logging
import re
import collections
import networkx as nx
sk.__version__

'0.20.1'

In [150]:
node_info = pd.read_csv("./node_information.csv", names=["ID","year","title","authors","journal","abstracts"])
data = np.loadtxt("./training_set.txt")

In [151]:
X, Y = data[:,0:-1], data[:,-1]
n = len(Y)
#X_pd = pd.DataFrame(X.astype(int),columns=["ID1", "ID2"])
X_pd = pd.read_csv("./train_pd.csv",index_col=0)

In [152]:
X_pd.columns

Index(['ID1', 'ID2', 'year1', 'year2', 'pos1', 'pos2', 'simi_abst',
       'author_overlap', 'title_overlap', 'journal_overlap',
       'journal_overlap_original', 'neighbors_1', 'neighbors_2',
       'common_neighbors', 'jaccard', 'resource_allocation', 'connectivity',
       'preferential_attachement', 'community1', 'community2',
       'same_community'],
      dtype='object')

In [153]:
columns = ["year1","year2","simi_abst","author_overlap","title_overlap","journal_overlap",
          "journal_overlap_original", "neighbors_1","neighbors_2","common_neighbors",
          "jaccard", "resource_allocation","connectivity","preferential_attachement","same_community"]

X_values = X_pd.loc[:,columns].values
X_train, X_test, y_train, y_test = train_test_split(X_values,
                                                    Y[:], test_size=0.2, random_state=42)
data_test = np.loadtxt("./testing_set.txt")
n_test = len(data_test)
test_pd = pd.read_csv("./test_pd.csv",index_col=0)
test_X = test_pd.loc[:,columns].values

In [167]:
scaler = preprocessing.StandardScaler()
X_values = scaler.fit_transform(X_values)

test_X = scaler.transform(test_X)


In [154]:
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [168]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier((128,64,32,16), max_iter = 1000)
clf.fit(X_values, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(128, 64, 32, 16), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [169]:
clf.score(X_values, Y)

0.97691352889951777

In [170]:
y_pred = clf.predict(X_values)
f1_score(Y,y_pred)

0.97871830585134256

In [171]:
y_pred = clf.predict(test_X)
pred_pd = pd.DataFrame(y_pred.astype(int), columns=["category"])
pred_pd.to_csv('pred_nn.csv', index = True,index_label = "id")

In [9]:
y_pred = clf.predict(X_test)
f1_score(y_test,y_pred)

0.97198413713700904

In [ ]:
clf.score(X_values, Y)

In [ ]:
y_pred = clf.predict(X_values)
f1_score(Y,y_pred)

In [ ]:
y_pred = clf.predict(test_X)
pred_pd = pd.DataFrame(y_pred.astype(int), columns=["category"])
pred_pd.to_csv('pred4.csv', index = True,index_label = "id")

In [ ]:
np.mean(y_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
forest.fit(X_values, Y)

In [ ]:
forest.score(X_values, Y)

In [ ]:
y_pred = forest.predict(X_values)
f1_score(Y,y_pred)

In [ ]:
y_pred = forest.predict(test_X)
pred_pd = pd.DataFrame(y_pred.astype(int), columns=["category"])
pred_pd.to_csv('pred3.csv', index = True,index_label = "id")

In [ ]:
sk.__version__

In [29]:
import xgboost
xgboost.__version__

'0.72.1'

In [160]:
scaler = preprocessing.StandardScaler()
X_values = scaler.fit_transform(X_values)

test_X = scaler.transform(test_X)

In [155]:
scaler = scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [161]:
xgb = xgboost.XGBClassifier(max_depth=8, n_estimators = 150)

In [163]:
xgb.fit(X_values,Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=150,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [164]:
xgb.score(X_values,Y)

0.97713610782568006

In [159]:
y_pred = xgb.predict(X_test)
f1_score(y_test,y_pred)

0.97640319017486055

In [165]:
y_pred = xgb.predict(test_X)
# f1_score(y_test,y_pred)

In [166]:
pred_pd = pd.DataFrame(y_pred.astype(int), columns=["category"])
pred_pd.to_csv('pred_xgb_150_8.csv', index = True,index_label = "id")

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
n_estimators = [100, 150, 200 , 250]
max_depth = [2, 4, 6, 8, 10, 12]
print(max_depth)
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

In [16]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier((256,128,64,32,16), max_iter = 1000)
nn.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(256, 128, 64, 32, 16), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [17]:
nn.score(X_train,y_train)

0.97808326005414203

In [20]:
y_pred = nn.predict(X_train)
f1_score(y_train,y_pred)

0.97962797265104395

In [21]:
y_pred = nn.predict(X_test)
f1_score(y_test,y_pred)

0.97054924527733433

In [148]:
import community

ModuleNotFoundError: No module named 'community'